# Library

In [47]:
import pandas as pd
import pickle
import pandas as pd
pd.options.plotting.backend = "plotly"
import util
from sklearn.metrics import mean_squared_error
import pickle
from sklearn.ensemble import RandomForestRegressor

# Load data and set index

In [48]:
dataset = pd.read_csv('../data_train\df_raw.csv')
dataset.index = dataset.time
dataset =dataset.drop(['time'],axis=1)
dataset

,temperature,co2,iaq,deviceId
time,,,,
2022-11-07 23:15:18+11:00,24.787203,1063.735229,179.0,E7184ABA7A055FD1
2022-11-07 23:15:25+11:00,23.957020,1065.705811,179.0,7BB92D02D696C5C5
2022-11-07 23:15:55+11:00,24.142492,651.757324,79.0,99DD33EFB7990A71
2022-11-07 23:15:57+11:00,23.676592,1170.204346,198.0,1DA36D3BE34D559A
2022-11-07 23:16:07+11:00,24.771078,1066.842407,179.0,B84C4503F361D64A
...,...,...,...,...
2022-11-11 13:59:16+11:00,23.982397,801.501953,119.0,7BB92D02D696C5C5
2022-11-11 13:59:39+11:00,24.150066,617.686890,60.0,99DD33EFB7990A71
2022-11-11 13:59:47+11:00,24.403280,713.524902,98.0,1DA36D3BE34D559A


# Seperate data by sensors id

In [49]:
dfs = util.df_group_by_col(df=dataset, col='deviceId')
dataset = dfs[0]
dataset

,temperature,co2,iaq,deviceId
time,,,,
2022-11-07 23:15:18+11:00,24.787203,1063.735229,179.0,E7184ABA7A055FD1
2022-11-07 23:16:18+11:00,24.782167,1056.472656,177.0,E7184ABA7A055FD1
2022-11-07 23:17:18+11:00,24.789717,1065.587769,179.0,E7184ABA7A055FD1
2022-11-07 23:18:18+11:00,24.784685,1064.388428,179.0,E7184ABA7A055FD1
2022-11-07 23:19:18+11:00,24.792234,1063.462036,178.0,E7184ABA7A055FD1
...,...,...,...,...
2022-11-11 13:56:08+11:00,24.724291,646.477234,76.0,E7184ABA7A055FD1
2022-11-11 13:57:08+11:00,24.704157,645.051270,75.0,E7184ABA7A055FD1
2022-11-11 13:58:08+11:00,24.699125,653.881470,80.0,E7184ABA7A055FD1


# Setup Targets

In [50]:
# co2 as target
dataset['target'] = dataset['co2'].shift(-15)
dataset = dataset[:-15]
dataset_numeric= dataset.drop(['deviceId'], axis=1)
dataset_numeric

C:\Users\ANPCC\AppData\Local\Temp\ipykernel_36700\2727454598.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,temperature,co2,iaq,target
time,,,,
2022-11-07 23:15:18+11:00,24.787203,1063.735229,179.0,1042.386719
2022-11-07 23:16:18+11:00,24.782167,1056.472656,177.0,1056.373047
2022-11-07 23:17:18+11:00,24.789717,1065.587769,179.0,1062.436279
2022-11-07 23:18:18+11:00,24.784685,1064.388428,179.0,1049.018311
2022-11-07 23:19:18+11:00,24.792234,1063.462036,178.0,1053.749268
...,...,...,...,...
2022-11-11 13:41:08+11:00,24.736872,613.454834,57.0,646.477234
2022-11-11 13:42:08+11:00,24.754488,615.277893,58.0,645.051270
2022-11-11 13:43:08+11:00,24.714224,636.333313,70.0,653.881470


# Data scaling - Normalization

In [51]:
dataset_scaled =(dataset_numeric-dataset_numeric.min())/(dataset_numeric.max()-dataset_numeric.min())
dataset_scaled

,temperature,co2,iaq,target
time,,,,
2022-11-07 23:15:18+11:00,0.522236,0.613637,0.842105,0.585213
2022-11-07 23:16:18+11:00,0.519366,0.603967,0.828947,0.603834
2022-11-07 23:17:18+11:00,0.523670,0.616103,0.842105,0.611907
2022-11-07 23:18:18+11:00,0.520801,0.614506,0.842105,0.594042
2022-11-07 23:19:18+11:00,0.525105,0.613273,0.835526,0.600341
...,...,...,...,...
2022-11-11 13:41:08+11:00,0.493541,0.014121,0.039474,0.058088
2022-11-11 13:42:08+11:00,0.503585,0.016548,0.046053,0.056189
2022-11-11 13:43:08+11:00,0.480629,0.044582,0.125000,0.067946


## De-normalization factors

In [52]:
max_d = dataset_numeric['target'].max()
min_d = dataset_numeric['target'].min()
#denormalized_d = normalized_d * (max_d - min_d) + min_d
print(max_d, min_d)

1353.922607421875 602.8489379882812


# Create lookback feature

In [53]:
lookback = 60

features = list(dataset_scaled.drop(['target'],axis=1).columns)
dataset_lb = dataset_scaled[:]

for i in range(1,lookback):    
    features_lb = list(map(lambda ls: ls+"_"+str(i), features))
    dataset_lb[features_lb]= dataset_scaled.drop(['target'],axis=1).shift(i)

dataset_lb = dataset_lb[lookback:]
dataset_lb
dataset = dataset_lb
dataset

C:\Users\ANPCC\AppData\Local\Temp\ipykernel_36700\223088852.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ANPCC\AppData\Local\Temp\ipykernel_36700\223088852.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ANPCC\AppData\Local\Temp\ipykernel_36700\223088852.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

,temperature,co2,iaq,target,temperature_1,co2_1,iaq_1,temperature_2,co2_2,iaq_2,...,iaq_56,temperature_57,co2_57,iaq_57,temperature_58,co2_58,iaq_58,temperature_59,co2_59,iaq_59
time,,,,,,,,,,,,,,,,,,,,,
2022-11-08 00:15:18+11:00,0.487803,0.589232,0.809211,0.551340,0.487803,0.588067,0.809211,0.484933,0.588588,0.809211,...,0.835526,0.520801,0.614506,0.842105,0.523670,0.616103,0.842105,0.519366,0.603967,0.828947
2022-11-08 00:16:18+11:00,0.479212,0.582024,0.802632,0.582618,0.487803,0.589232,0.809211,0.487803,0.588067,0.809211,...,0.835526,0.525105,0.613273,0.835526,0.520801,0.614506,0.842105,0.523670,0.616103,0.842105
2022-11-08 00:17:18+11:00,0.479194,0.587984,0.809211,0.581631,0.479212,0.582024,0.802632,0.487803,0.589232,0.809211,...,0.835526,0.519366,0.609991,0.835526,0.525105,0.613273,0.835526,0.520801,0.614506,0.842105
2022-11-08 00:18:18+11:00,0.477759,0.582083,0.802632,0.588554,0.479194,0.587984,0.809211,0.479212,0.582024,0.802632,...,0.835526,0.516497,0.612629,0.835526,0.519366,0.609991,0.835526,0.525105,0.613273,0.835526
2022-11-08 00:19:18+11:00,0.479212,0.582584,0.802632,0.578412,0.477759,0.582083,0.802632,0.479194,0.587984,0.809211,...,0.835526,0.510757,0.610349,0.835526,0.516497,0.612629,0.835526,0.519366,0.609991,0.835526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-11 13:41:08+11:00,0.493541,0.014121,0.039474,0.058088,0.490671,0.013907,0.039474,0.476325,0.013700,0.039474,...,0.092105,0.407459,0.032102,0.092105,0.406023,0.034517,0.098684,0.390241,0.033114,0.092105
2022-11-11 13:42:08+11:00,0.503585,0.016548,0.046053,0.056189,0.493541,0.014121,0.039474,0.490671,0.013907,0.039474,...,0.078947,0.408892,0.031365,0.092105,0.407459,0.032102,0.092105,0.406023,0.034517,0.098684
2022-11-11 13:43:08+11:00,0.480629,0.044582,0.125000,0.067946,0.503585,0.016548,0.046053,0.493541,0.014121,0.039474,...,0.085526,0.408892,0.026473,0.078947,0.408892,0.031365,0.092105,0.407459,0.032102,0.092105


# Train valid test split

In [54]:
val_split = int(len(dataset) * 0.7)

data_train = dataset[:val_split]
data_validation = dataset[val_split:]

test_split = int(len(data_validation) * 0.5)

data_test = data_validation[test_split:]
data_validation = data_validation[:test_split]

#===============================================================

print(data_train.shape, data_validation.shape ,data_test.shape )
data_train.head()

(3309, 181) (709, 181) (710, 181)


,temperature,co2,iaq,target,temperature_1,co2_1,iaq_1,temperature_2,co2_2,iaq_2,...,iaq_56,temperature_57,co2_57,iaq_57,temperature_58,co2_58,iaq_58,temperature_59,co2_59,iaq_59
time,,,,,,,,,,,,,,,,,,,,,
2022-11-08 00:15:18+11:00,0.487803,0.589232,0.809211,0.551340,0.487803,0.588067,0.809211,0.484933,0.588588,0.809211,...,0.835526,0.520801,0.614506,0.842105,0.523670,0.616103,0.842105,0.519366,0.603967,0.828947
2022-11-08 00:16:18+11:00,0.479212,0.582024,0.802632,0.582618,0.487803,0.589232,0.809211,0.487803,0.588067,0.809211,...,0.835526,0.525105,0.613273,0.835526,0.520801,0.614506,0.842105,0.523670,0.616103,0.842105
2022-11-08 00:17:18+11:00,0.479194,0.587984,0.809211,0.581631,0.479212,0.582024,0.802632,0.487803,0.589232,0.809211,...,0.835526,0.519366,0.609991,0.835526,0.525105,0.613273,0.835526,0.520801,0.614506,0.842105
2022-11-08 00:18:18+11:00,0.477759,0.582083,0.802632,0.588554,0.479194,0.587984,0.809211,0.479212,0.582024,0.802632,...,0.835526,0.516497,0.612629,0.835526,0.519366,0.609991,0.835526,0.525105,0.613273,0.835526
2022-11-08 00:19:18+11:00,0.479212,0.582584,0.802632,0.578412,0.477759,0.582083,0.802632,0.479194,0.587984,0.809211,...,0.835526,0.510757,0.610349,0.835526,0.516497,0.612629,0.835526,0.519366,0.609991,0.835526


# Final feature selection

In [55]:
X_train = data_train.drop(['target'],axis=1).astype("float64")
X_val = data_validation.drop(['target'],axis=1).astype("float64")
X_test = data_test.drop(['target'],axis=1).astype("float64")

y_train = data_train[["target"]].astype("float64")
y_val = data_validation[["target"]].astype("float64")
y_test = data_test[["target"]].astype("float64")

num_feature = X_train.shape[1]
    
print(X_train.shape, X_val.shape,X_test.shape)
print(y_train.shape,y_val.shape,y_test.shape)
print(num_feature)


(3309, 180) (709, 180) (710, 180)
(3309, 1) (709, 1) (710, 1)
180


# Define model

In [56]:
model = RandomForestRegressor(max_depth=100, random_state=0)

# Training

In [57]:
model.fit(X_train, y_train)

C:\Users\ANPCC\AppData\Local\Temp\ipykernel_36700\180087699.py:1: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RandomForestRegressor(max_depth=100, random_state=0)

# Inference

In [58]:
# Predict with the best model(includes original training data).
y_pred = model.predict(X_test)
y_pred

array([0.18985402, 0.20207094, 0.24158167, 0.20316731, 0.2384573 ,
       0.21356653, 0.25519767, 0.26173773, 0.26204579, 0.25481104,
       0.25066484, 0.25269042, 0.27176268, 0.27152168, 0.27822441,
       0.27567623, 0.27767416, 0.27448734, 0.26467445, 0.26453215,
       0.25768875, 0.25477532, 0.25648866, 0.27366907, 0.274141  ,
       0.27674711, 0.2672778 , 0.27941703, 0.27100081, 0.27967302,
       0.28103315, 0.27344011, 0.28310256, 0.28789271, 0.28018799,
       0.27969523, 0.27917038, 0.2777598 , 0.28204392, 0.28057929,
       0.27913599, 0.28253909, 0.2876149 , 0.28173285, 0.27981678,
       0.2796456 , 0.2786585 , 0.2769572 , 0.27485413, 0.27865629,
       0.27594224, 0.27698437, 0.27719436, 0.29649411, 0.28667379,
       0.28475473, 0.27859645, 0.27732354, 0.30428275, 0.30206053,
       0.30221653, 0.30921249, 0.29010078, 0.2815001 , 0.30395783,
       0.31441522, 0.30450327, 0.32239022, 0.31869054, 0.31790996,
       0.33441055, 0.33931803, 0.34100847, 0.34235204, 0.34388

# Evaluation

In [59]:
pred_val =model.predict(X_val) * (max_d - min_d)+ min_d
true_val =y_val * (max_d - min_d) + min_d

pred_test =model.predict(X_test) * (max_d - min_d)+ min_d
true_test = y_test * (max_d - min_d) + min_d

pred_train =model.predict(X_train) * (max_d - min_d)+ min_d
true_train = y_train * (max_d - min_d) + min_d

print(mean_squared_error(model.predict(X_val), y_val))
print(mean_squared_error(model.predict(X_test), y_test))

print(mean_squared_error(pred_train , true_train))
print(mean_squared_error(pred_val , true_val))
print(mean_squared_error(pred_test , true_test))

0.0007888453573446075
0.002623064207446535
19.789303641747242
444.9968615825102
1479.7010962608817


# Visualize Result

In [60]:
df_viz_train = pd.DataFrame()
df_viz_train['pred_train'] =pred_train.reshape(-1)
df_viz_train['true_train'] =true_train.values
df_viz_train.plot()

In [61]:
df_viz_val = pd.DataFrame()
df_viz_val['pred_val'] =pred_val.reshape(-1)
df_viz_val['true_val'] =true_val.values
df_viz_val.plot()

In [62]:
df_viz_test = pd.DataFrame()
df_viz_test['pred_test'] =pred_test.reshape(-1)
df_viz_test['true_test'] =true_test.values
df_viz_test.plot()

# Save model

In [63]:
file_name = "../saved_model/rf_reg.pkl"
# save
pickle.dump(model, open(file_name, "wb"))

# Load model

In [64]:
# load
model_loaded = pickle.load(open(file_name, "rb"))

# Load model Inference

In [65]:
result = model_loaded.predict(X_test)
result_denor = result  * (max_d - min_d) + min_d
result_denor

array([745.44329285, 754.61910156, 784.29456665, 755.44255737,
       781.94793823, 763.25313232, 794.5211908 , 799.43325256,
       799.66463013, 794.23079834, 791.11669617, 792.63805725,
       806.96273132, 806.78171936, 811.81596436, 809.90209717,
       811.40268982, 809.00915344, 801.63894592, 801.53207214,
       796.39217468, 794.20396973, 795.49081787, 808.39457092,
       808.7490271 , 810.70640198, 803.59425781, 812.71171387,
       806.39051453, 812.90398132, 813.92553345, 808.22260742,
       815.47981567, 819.07756897, 813.29076294, 812.9206604 ,
       812.5264624 , 811.46701233, 814.68470276, 813.58465637,
       812.50062927, 815.05660828, 818.86891785, 814.45106567,
       813.01195557, 812.88338562, 812.14200378, 810.86419861,
       809.28463562, 812.14034119, 810.10188904, 810.88460388,
       811.04232605, 825.53786072, 818.16207642, 816.72071472,
       812.09539307, 811.13934448, 831.3877002 , 829.71865173,
       829.83581421, 835.09029907, 820.73599182, 814.27